# 基于带货视频评论的用户洞察分析——基于科大讯飞数据挖掘应用赛

## 项目介绍

在电商直播爆发式增长的数字化浪潮下，短视频平台积累了海量带货视频及用户互动数据。在此背景下，通过智能识别视频中推广的核心商品，结合评论区用户的情感表达与观点聚合，企业能够精准捕捉消费者对商品的真实态度与需求痛点。这种分析方式不仅能揭示用户对商品功能、价格、服务的多维评价，还可通过情感倾向聚类，构建消费者偏好画像，为选品策略优化和网红合作评估提供数据支撑。
赛事聚焦"商品识别-情感分析-聚类洞察"的完整链条：先基于视频内容建立商品关联关系，进而从非结构化评论中提取情感倾向，最终通过聚类总结形成结构化洞察。这一研究路径既可帮助品牌穿透数据迷雾把握消费心理，又能科学评估网红的内容种草效果与带货转化潜力，实现从内容营销到消费决策的全链路价值提升。

## 任务拆解

基于官方提供的带货视频文本及评论文本数据，完成以下三阶段分析任务：

- 【商品识别】精准识别推广商品；

- 【情感分析】对评论文本进行多维度情感分析，涵盖维度见数据说明；

- 【评论聚类】按商品对归属指定维度的评论进行聚类，并提炼类簇总结词。

## 使用数据——来自科大讯飞官方
85条脱敏后的带货视频数据及6477条评论文本数据，数据包括少量有人工标注结果的训练集（仅包含商品识别和情感分析的标注结果）以及未标注的测试集。所有数据均经过脱敏处理，确保信息安全，其格式说明如下：

- 带货视频内容文本信息的数据格式

| 序号 | 变量名称     | 变量格式 | 解释         |
| ---- | ------------ | -------- | ------------ |
| 1    | video_id     | string   | 视频id       |
| 2    | video_desc   | string   | 视频描述     |
| 3    | video_tags   | string   | 视频标签     |
| 4    | product_name | string   | 推广商品名称 |

注：product_name需根据提供的视频信息进行提取，并从匹配到商品列表**[Xfaiyx Smart Translator, Xfaiyx Smart Recorder]**中的一项。

- 评论区文本信息的数据格式

| 序号 | 变量名称                 | 变量格式 | 解释                                 |
| ---- | ------------------------ | -------- | ------------------------------------ |
| 1    | video_id                 | string   | 视频id                               |
| 2    | comment_id               | string   | 评论id                               |
| 3    | comment_text             | string   | 评论文本                             |
| 4    | sentiment_category       | int      | 关于商品的情感倾向分类               |
| 5    | user_scenario            | int      | 是否与用户场景有关，0表示否，1表示是 |
| 6    | user_question            | int      | 是否与用户疑问有关，0表示否，1表示是 |
| 7    | user_suggestion          | int      | 是否与用户建议有关，0表示否，1表示是 |
| 8    | positive_cluster_theme   | string   | 按正面倾向聚类的类簇主题词           |
| 9    | negative_cluster_theme   | string   | 按负面倾向聚类的类簇主题词           |
| 10   | scenario_cluster_theme   | string   | 按用户场景聚类的类簇主题词           |
| 11   | question_cluster_theme   | string   | 按用户疑问聚类的类簇主题词           |
| 12   | suggestion_cluster_theme | string   | 按用户建议聚类的类簇主题词           |

注：

a. 需进行情感分析的字段包括sentiment_category、user_scenario、user_question和user_suggestion。训练集中部分数据已提供标注，测试集需自行预测。其中字段sentiment_category情感倾向分类的数值含义见下表：

| 分类值 | 1    | 2    | 3          | 4    | 5      |
| ------ | ---- | ---- | ---------- | ---- | ------ |
| 含义   | 正面 | 负面 | 正负都包含 | 中性 | 不相关 |

b. 需进行聚类的字段包括：

- positive_cluster_theme：基于训练集和测试集中正面倾向（sentiment_category=1 或 sentiment_category=3）的评论进行聚类并提炼主题词。
- negative_cluster_theme：基于训练集和测试集中负面倾向（sentiment_category=2 或 sentiment_category=3）的评论进行聚类并提炼主题词。
- scenario_cluster_theme：基于训练集和测试集中用户场景相关评论（user_scenario=1）进行聚类并提炼主题词。
- question_cluster_theme：基于训练集和测试集中用户疑问相关评论（user_question=1）进行聚类并提炼主题词。
- suggestion_cluster_theme：基于训练集和测试集中用户建议相关评论（user_suggestion=1）进行聚类并提炼主题词。

**注意，聚类样本包含训练集和测试集的全部满足上述条件的评论样本。**

## 解决方法
- 使用工具：ModelScope
- 基于向量化的用户评论数据运用fit预测每条评论对应的产品、情感倾向类别（如建议、疑惑等）
- 构建了基于TF-IDF和K-Means的文本聚类流水线，自动提取每一类情感倾向评论对应的主题关键词，挖掘用户情感背后的原因。
- 优化：采用jieba分词和k-means++初始化优化中文文本处理，通过轮廓系数动态选择最佳聚类数（k=2~7），最终评论解释得到50%的改善。


In [1]:
import pandas as pd
video_data = pd.read_csv("origin_videos_data.csv")
comments_data = pd.read_csv("origin_comments_data.csv")

In [2]:
video_data.sample(10)

,video_id,video_desc,video_tags,product_name
11,v9edfKB,Xfaiyx Übersetzer: Mein unverzichtbares Gadget...,Handson;;Hands-On;;Translator;;Übersetzer;;Spr...,Xfaiyx Smart Translator
82,vnK5t9y,Q:「外国人の彼女欲しいけど英語話せません…」#XfaiyxSmartTranslator ...,海外の反応;;外国人の反応;;ロシア人の反応;;外人;;外国人;;外国人 日本;;外国人 日...,NaN
48,vdbLgWS,WE FOUND A SOLUTION TO OUR PROBLEM! ft Xfaiyx ...,NaN,NaN
58,vmsHv2Z,This thing could mess up some relationships 😳👀...,amazonfinds;;amazontechfinds;;amazon;;tech;;te...,NaN
56,vJAQbN9,Sprachübersetzer von Xfaiyx im Test 📱 Anzeige ...,übersetzer;;sprachen;;Xfaiyx,NaN
21,vhLcap1,"60 Languages TRANSLATED Instantly, fits in YOU...",Xfaiyx;;Xfaiyx Translator;;Instant Voice Trans...,NaN
72,vZBnD1J,"Break Language Barriers, Not Your Journey. Spe...",Xfaiyx;;Xfaiyxtranslator;;instantvoicetranslat...,NaN
0,vmCnKGQ,Xfaiyx Smart Recorder Product Review,Muzain Reviews;;Gadget;;Technology;;Reviews;;U...,Xfaiyx Smart Recorder
84,v2cabia,Living Alone Diaries | A Realistic Day In My L...,NaN,NaN
60,vgIkaeR,ESCAPE THE BUSY LONDON AND GO ON A DAY TRIP TO...,vlog;;nhan ta;;nhân;;nhan's diaries;;living al...,NaN


In [16]:
comments_data

,video_id,comment_id,comment_text,sentiment_category,user_scenario,user_question,user_suggestion,positive_cluster_theme,negative_cluster_theme,scenario_cluster_theme,question_cluster_theme,suggestion_cluster_theme
0,v8TdcdK,c-0KPeWgS,Pro just develop bro codes,5.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,vB3iY9I,c-0uMbk_C,Frivolous lawsuits are against the law!!!!!<br...,5.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,v8TdcdK,c-4-XnVYb,Part 2 two pls I want to know what there punis...,5.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,v8TdcdK,c-7CvRdAr,I tried using the Xfaiyx Translator during my ...,1.0,1.0,0.0,0.0,NaN,NaN,"for, it, i, the, .",NaN,NaN
4,vWN5LPe,c-9SGtvvf,I’ve been using the Xfaiyx Smart Translator du...,1.0,1.0,0.0,0.0,NaN,NaN,"for, it, i, the, .",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6471,vOREHMM,cznE14nWP,Is it suitable for business meetings?,4.0,1.0,1.0,0.0,NaN,NaN,"for, it, i, the, .","the, to, is, it, what",NaN
6472,vJAQbN9,czp-Lr69-,Ich spreche deutsch englisch und chinesisch. B...,2.0,0.0,0.0,0.0,NaN,"ü, die, ä, ich, ist",NaN,NaN,NaN
6473,vdzrIP2,czsum0ump,翻訳機を使った結果が少し直訳的で意味が通じない部分がありました。次のバージョンに期待しています。,5.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
6474,vZNFtEb,cztffSUXy,이 번역기가 실시간 번역 속도가 빠른가요? 여행 중에 바로바로 사용할 수 있는지 궁...,4.0,0.0,1.0,0.0,NaN,NaN,NaN,"can, or, handle, translate, it",NaN


In [5]:
video_data["text"] = video_data["video_desc"].fillna("") + " " + video_data["video_tags"].fillna("")

In [6]:
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline

In [ ]:
product_name_predictor = make_pipeline(
    TfidfVectorizer(tokenizer=jieba.lcut, max_features=50), SGDClassifier()
)
product_name_predictor.fit(
    video_data[~video_data["product_name"].isnull()]["text"],
    video_data[~video_data["product_name"].isnull()]["product_name"],
)
video_data["product_name"] = product_name_predictor.predict(video_data["text"])

In [9]:
comments_data.columns

Index(['video_id', 'comment_id', 'comment_text', 'sentiment_category',
       'user_scenario', 'user_question', 'user_suggestion',
       'positive_cluster_theme', 'negative_cluster_theme',
       'scenario_cluster_theme', 'question_cluster_theme',
       'suggestion_cluster_theme'],
      dtype='object')

In [ ]:
for col in ['sentiment_category',
       'user_scenario', 'user_question', 'user_suggestion']:
    predictor = make_pipeline(
        TfidfVectorizer(tokenizer=jieba.lcut), SGDClassifier()
    )
    predictor.fit(
        comments_data[~comments_data[col].isnull()]["comment_text"],
        comments_data[~comments_data[col].isnull()][col],
    )
    comments_data[col] = predictor.predict(comments_data["comment_text"])

In [11]:
top_n_words = 10

In [17]:
import jieba
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# 1. 数据准备
negative_comments = comments_data[comments_data["sentiment_category"].isin([1, 3])]
texts = negative_comments["comment_text"].fillna("")  # 处理空值

# 2. 动态选择最佳聚类数
best_k, best_score = 0, -1
for k in range(2, 8):  # 尝试 k=2~7
    pipeline = make_pipeline(
        TfidfVectorizer(tokenizer=jieba.lcut, max_df=0.8, min_df=2),
        KMeans(n_clusters=k, init='k-means++', n_init=10)
    )
    labels = pipeline.fit_predict(texts)
    score = silhouette_score(pipeline.named_steps['tfidfvectorizer'].transform(texts), labels)
    if score > best_score:
        best_k, best_score = k, score

# 3. 用最佳 k 重新聚类并提取主题
final_pipeline = make_pipeline(
    TfidfVectorizer(tokenizer=jieba.lcut, max_df=0.8, min_df=2),
    KMeans(n_clusters=best_k, init='k-means++', n_init=10)
)
final_pipeline.fit(texts)
cluster_labels = final_pipeline.predict(texts)

# 4. 提取主题关键词
tfidf = final_pipeline.named_steps['tfidfvectorizer']
kmeans = final_pipeline.named_steps['kmeans']
feature_names = tfidf.get_feature_names_out()
top_n_words = 5  # 每簇取前5个关键词

cluster_themes = []
for i in range(best_k):
    top_words_idx = kmeans.cluster_centers_[i].argsort()[::-1][:top_n_words]
    cluster_themes.append(", ".join(feature_names[top_words_idx]))

# 5. 保存结果
comments_data.loc[negative_comments.index, "negative_cluster_theme"] = [cluster_themes[label] for label in cluster_labels]

/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:526: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/feature_extr

In [ ]:
import jieba
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# 1. 数据准备
negative_comments = comments_data[comments_data["sentiment_category"].isin([4, 5])]
texts = negative_comments["comment_text"].fillna("")  # 处理空值

# 2. 动态选择最佳聚类数
best_k, best_score = 0, -1
for k in range(2, 8):  # 尝试 k=2~7
    pipeline = make_pipeline(
        TfidfVectorizer(tokenizer=jieba.lcut, max_df=0.8, min_df=2),
        KMeans(n_clusters=k, init='k-means++', n_init=10)
    )
    labels = pipeline.fit_predict(texts)
    score = silhouette_score(pipeline.named_steps['tfidfvectorizer'].transform(texts), labels)
    if score > best_score:
        best_k, best_score = k, score

# 3. 用最佳 k 重新聚类并提取主题
final_pipeline = make_pipeline(
    TfidfVectorizer(tokenizer=jieba.lcut, max_df=0.8, min_df=2),
    KMeans(n_clusters=best_k, init='k-means++', n_init=10)
)
final_pipeline.fit(texts)
cluster_labels = final_pipeline.predict(texts)

# 4. 提取主题关键词
tfidf = final_pipeline.named_steps['tfidfvectorizer']
kmeans = final_pipeline.named_steps['kmeans']
feature_names = tfidf.get_feature_names_out()
top_n_words = 5  # 每簇取前5个关键词

cluster_themes = []
for i in range(best_k):
    top_words_idx = kmeans.cluster_centers_[i].argsort()[::-1][:top_n_words]
    cluster_themes.append(", ".join(feature_names[top_words_idx]))

# 5. 保存结果
comments_data.loc[negative_comments.index, "positive_cluster_theme"] = [cluster_themes[label] for label in cluster_labels]

In [ ]:
import jieba
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# 1. 数据准备
scenario_comments = comments_data[comments_data["user_scenario"].isin([1])]
texts = scenario_comments["comment_text"].fillna("")  # 处理空值

# 2. 动态选择最佳聚类数
best_k, best_score = 0, -1
for k in range(2, 7):  # 尝试 k=2~7
    pipeline = make_pipeline(
        TfidfVectorizer(tokenizer=jieba.lcut, max_df=0.8, min_df=2),
        KMeans(n_clusters=k, init='k-means++', n_init=10)
    )
    labels = pipeline.fit_predict(texts)
    score = silhouette_score(pipeline.named_steps['tfidfvectorizer'].transform(texts), labels)
    if score > best_score:
        best_k, best_score = k, score

# 3. 用最佳 k 重新聚类并提取主题
final_pipeline = make_pipeline(
    TfidfVectorizer(tokenizer=jieba.lcut, max_df=0.8, min_df=2),
    KMeans(n_clusters=best_k, init='k-means++', n_init=10)
)
final_pipeline.fit(texts)
cluster_labels = final_pipeline.predict(texts)

# 4. 提取主题关键词
tfidf = final_pipeline.named_steps['tfidfvectorizer']
kmeans = final_pipeline.named_steps['kmeans']
feature_names = tfidf.get_feature_names_out()
top_n_words = 5  # 每簇取前5个关键词

cluster_themes = []
for i in range(best_k):
    top_words_idx = kmeans.cluster_centers_[i].argsort()[::-1][:top_n_words]
    cluster_themes.append(", ".join(feature_names[top_words_idx]))

# 5. 保存结果
comments_data.loc[scenario_comments.index, "scenario_cluster_theme"] = [cluster_themes[label] for label in cluster_labels]

In [ ]:
import jieba
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# 1. 数据准备
question_comments = comments_data[comments_data["user_question"].isin([1])]
texts = question_comments["comment_text"].fillna("")  # 处理空值

# 2. 动态选择最佳聚类数
best_k, best_score = 0, -1
for k in range(2, 8):  # 尝试 k=2~7
    pipeline = make_pipeline(
        TfidfVectorizer(tokenizer=jieba.lcut, max_df=0.8, min_df=2),
        KMeans(n_clusters=k, init='k-means++', n_init=10)
    )
    labels = pipeline.fit_predict(texts)
    score = silhouette_score(pipeline.named_steps['tfidfvectorizer'].transform(texts), labels)
    if score > best_score:
        best_k, best_score = k, score

# 3. 用最佳 k 重新聚类并提取主题
final_pipeline = make_pipeline(
    TfidfVectorizer(tokenizer=jieba.lcut, max_df=0.8, min_df=2),
    KMeans(n_clusters=best_k, init='k-means++', n_init=10)
)
final_pipeline.fit(texts)
cluster_labels = final_pipeline.predict(texts)

# 4. 提取主题关键词
tfidf = final_pipeline.named_steps['tfidfvectorizer']
kmeans = final_pipeline.named_steps['kmeans']
feature_names = tfidf.get_feature_names_out()
top_n_words = 5  # 每簇取前5个关键词

cluster_themes = []
for i in range(best_k):
    top_words_idx = kmeans.cluster_centers_[i].argsort()[::-1][:top_n_words]
    cluster_themes.append(", ".join(feature_names[top_words_idx]))

# 5. 保存结果
comments_data.loc[question_comments.index, "question_cluster_theme"] = [cluster_themes[label] for label in cluster_labels]

In [ ]:
import jieba
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# 1. 数据准备
question_comments = comments_data[comments_data["user_suggestion"].isin([1])]
texts = question_comments["comment_text"].fillna("")  # 处理空值

# 2. 动态选择最佳聚类数
best_k, best_score = 0, -1
for k in range(2, 8):  # 尝试 k=2~7
    pipeline = make_pipeline(
        TfidfVectorizer(tokenizer=jieba.lcut, max_df=0.8, min_df=2),
        KMeans(n_clusters=k, init='k-means++', n_init=10)
    )
    labels = pipeline.fit_predict(texts)
    score = silhouette_score(pipeline.named_steps['tfidfvectorizer'].transform(texts), labels)
    if score > best_score:
        best_k, best_score = k, score

# 3. 用最佳 k 重新聚类并提取主题
final_pipeline = make_pipeline(
    TfidfVectorizer(tokenizer=jieba.lcut, max_df=0.8, min_df=2),
    KMeans(n_clusters=best_k, init='k-means++', n_init=10)
)
final_pipeline.fit(texts)
cluster_labels = final_pipeline.predict(texts)

# 4. 提取主题关键词
tfidf = final_pipeline.named_steps['tfidfvectorizer']
kmeans = final_pipeline.named_steps['kmeans']
feature_names = tfidf.get_feature_names_out()
top_n_words = 5  # 每簇取前5个关键词

cluster_themes = []
for i in range(best_k):
    top_words_idx = kmeans.cluster_centers_[i].argsort()[::-1][:top_n_words]
    cluster_themes.append(", ".join(feature_names[top_words_idx]))

# 5. 保存结果
comments_data.loc[question_comments.index, "suggestion_cluster_theme"] = [cluster_themes[label] for label in cluster_labels]

In [20]:
!rm -rf submit 
!mkdir submit

In [21]:
video_data[["video_id", "product_name"]].to_csv("submit/submit_videos.csv", index=None)
comments_data[['video_id', 'comment_id', 'sentiment_category',
       'user_scenario', 'user_question', 'user_suggestion',
       'positive_cluster_theme', 'negative_cluster_theme',
       'scenario_cluster_theme', 'question_cluster_theme',
       'suggestion_cluster_theme']].to_csv("submit/submit_comments.csv", index=None)

In [16]:
!zip -r submit.zip submit/

updating: submit/ (stored 0%)
updating: submit/submit_comments.csv (deflated 78%)
updating: submit/submit_videos.csv (deflated 78%)
